In [1]:
from typing import Tuple, List, Union, Any, Optional, Dict, Literal, Callable, Type
import abc

from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim
import torch.utils.data
from torch import Tensor


from models.ridge_ALOOCV import fit_ridge_ALOOCV
from models.sandwiched_least_squares import sandwiched_LS_dense, sandwiched_LS_diag, sandwiched_LS_scalar

In [2]:
# Make regression data X, y
N = 1000
N_test = 1000
D = 10
d = 2
X = torch.randn(N, D)
X_test = torch.randn(N_test, D)
w_true = torch.randn(D, d)
y = (X @ w_true)**2 + torch.randn(N, d) * 0.1  # Adding some noise
y_test = (X_test @ w_true)**2 + torch.randn(N_test, d) * 0.1  # Adding some noise

In [ ]:
from models.base import RidgeCVModule

#dense      
model = RidgeCVModule(
        lower_alpha=1e-6,
        upper_alpha=1e6,
        n_alphas=10,
     )

results = []
for i in range(10):
    model.fit(X, y)
    out = model(X)
    out_test = model(X_test)
    rmse = torch.sqrt(nn.functional.mse_loss(out, y))
    rmse_test = torch.sqrt(nn.functional.mse_loss(out_test, y_test))
    results.append(torch.tensor([rmse, rmse_test]))
results = torch.stack(results)
print("train rmse", results[:, 0].mean(), "std", results[:, 0].std())
print("test rmse", results[:, 1].mean(), "std", results[:, 1].std())
print("train", results[:, 0])
print("test", results[:, 1])

In [ ]:
from models.base import RidgeModule


#dense      
model = RidgeModule(
        l2_reg = 1e+2,
     )

results = []
for i in range(10):
    model.fit(X, y)
    out = model(X)
    out_test = model(X_test)
    rmse = torch.sqrt(nn.functional.mse_loss(out, y))
    rmse_test = torch.sqrt(nn.functional.mse_loss(out_test, y_test))
    results.append(torch.tensor([rmse, rmse_test]))
results = torch.stack(results)
print("train rmse", results[:, 0].mean(), "std", results[:, 0].std())
print("test rmse", results[:, 1].mean(), "std", results[:, 1].std())
print("train", results[:, 0])
print("test", results[:, 1])

In [ ]:
from models.random_feature_representation_boosting import GreedyRFRBoostRegressor

model = GreedyRFRBoostRegressor(
        in_dim=D,
        out_dim=d,
        hidden_dim=128,
        n_layers=3,
        randfeat_xt_dim=128,
        randfeat_x0_dim=128,
        l2_reg=0.1,
        l2_ghat=0.001,
        boost_lr=1.0,
        sandwich_solver="dense",
        feature_type="SWIM",
        upscale_type="iid",
        use_batchnorm=True,
    )        

results = []
for i in range(10):
    model.fit(X, y)
    out = model(X)
    out_test = model(X_test)
    rmse = torch.sqrt(nn.functional.mse_loss(out, y))
    rmse_test = torch.sqrt(nn.functional.mse_loss(out_test, y_test))
    results.append(torch.tensor([rmse, rmse_test]))
results = torch.stack(results)
print("train rmse", results[:, 0].mean(), "std", results[:, 0].std())
print("test rmse", results[:, 1].mean(), "std", results[:, 1].std())
print("train", results[:, 0])
print("test", results[:, 1])

In [6]:
# train rmse tensor(2.3312) std tensor(0.5462)
# test rmse tensor(3.3370) std tensor(0.6041)
# train tensor([2.5459, 1.9770, 2.1338, 3.6567, 1.9788, 2.5264, 2.1827, 1.9025, 2.6022,
#         1.8058])
# test tensor([3.7197, 2.8087, 2.9656, 4.6065, 2.9207, 3.3118, 3.1447, 3.0470, 4.0548,
#         2.7906])

In [ ]:
from models.random_feature_representation_boosting import GreedyRFRBoostRegressor

model = GreedyRFRBoostRegressor(
        in_dim=D,
        out_dim=d,
        hidden_dim=128,
        n_layers=5,
        randfeat_xt_dim=128,
        randfeat_x0_dim=128,
        l2_reg=0.01,
        l2_ghat=0.001,
        boost_lr=1.0,
        sandwich_solver="diag",
        feature_type="SWIM",
        upscale_type="iid",
    )        

results = []
for i in range(10):
    model.fit(X, y)
    out = model(X)
    out_test = model(X_test)
    rmse = torch.sqrt(nn.functional.mse_loss(out, y))
    rmse_test = torch.sqrt(nn.functional.mse_loss(out_test, y_test))
    results.append(torch.tensor([rmse, rmse_test]))
results = torch.stack(results)
print("train rmse", results[:, 0].mean(), "std", results[:, 0].std())
print("test rmse", results[:, 1].mean(), "std", results[:, 1].std())
print("train", results[:, 0])
print("test", results[:, 1])

In [ ]:
from models.random_feature_representation_boosting import GreedyRFRBoostRegressor

model = GreedyRFRBoostRegressor(
        in_dim=D,
        out_dim=d,
        hidden_dim=128,
        n_layers=5,
        randfeat_xt_dim=128,
        randfeat_x0_dim=128,
        l2_reg=0.01,
        l2_ghat=0.000001,
        boost_lr=1.0,
        sandwich_solver="scalar",
        feature_type="SWIM",
        upscale_type="iid",
    )        

results = []
for i in range(10):
    model.fit(X, y)
    out = model(X)
    out_test = model(X_test)
    rmse = torch.sqrt(nn.functional.mse_loss(out, y))
    rmse_test = torch.sqrt(nn.functional.mse_loss(out_test, y_test))
    results.append(torch.tensor([rmse, rmse_test]))
results = torch.stack(results)
print("train rmse", results[:, 0].mean(), "std", results[:, 0].std())
print("test rmse", results[:, 1].mean(), "std", results[:, 1].std())
print("train", results[:, 0])
print("test", results[:, 1])

In [ ]:
from models.random_feature_representation_boosting import GradientRFRBoostRegressor

model = GradientRFRBoostRegressor(
        in_dim=D,
        out_dim=d,
        hidden_dim=128,
        n_layers=5,
        randfeat_xt_dim=128,
        randfeat_x0_dim=128,
        l2_reg=0.1,
        l2_ghat=0.000001,
        boost_lr=1.0,
        feature_type="SWIM",
        upscale_type="iid",
    )

results = []
for i in range(5):
    model.fit(X, y)
    out = model(X)
    out_test = model(X_test)
    rmse = torch.sqrt(nn.functional.mse_loss(out, y))
    rmse_test = torch.sqrt(nn.functional.mse_loss(out_test, y_test))
    results.append(torch.tensor([rmse, rmse_test]))
results = torch.stack(results)
print("train rmse", results[:, 0].mean(), "std", results[:, 0].std())
print("test rmse", results[:, 1].mean(), "std", results[:, 1].std())
print("train", results[:, 0])
print("test", results[:, 1])

# End2End

In [ ]:
from old.models import End2EndMLPResNet

model = End2EndMLPResNet(
        in_dim=D,
        hidden_dim=128,
        bottleneck_dim=32,
        out_dim=d,
        n_blocks=3,
        loss = "mse",
        lr = 0.1,
        n_epochs = 30,
        end_lr_factor= 0.1,
        weight_decay = 0.001,
        batch_size = 64,
        )

results = []
for i in range(5):
    model.fit(X, y)
    out = model(X)
    out_test = model(X_test)
    rmse = torch.sqrt(nn.functional.mse_loss(out, y))
    rmse_test = torch.sqrt(nn.functional.mse_loss(out_test, y_test))
    results.append(torch.tensor([rmse, rmse_test]))
results = torch.stack(results)
print("train rmse", results[:, 0].mean(), "std", results[:, 0].std())
print("test rmse", results[:, 1].mean(), "std", results[:, 1].std())
print("train", results[:, 0])
print("test", results[:, 1])